In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;
 

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [4]:
SELECT um.codmedicamento, COUNT(*) FROM UsoMedicamento um
    GROUP BY um.codmedicamento;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [5]:
-- criar view
CREATE VIEW UsoMedCount AS
SELECT um.codmedicamento, COUNT(*) medCount FROM UsoMedicamento um
    GROUP BY um.codmedicamento;

In [6]:
-- selecionar o maximo
SELECT MAX(medcount) FROM UsoMedCount


493

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [7]:
SELECT MAX(medCOUNT) FROM (
    SELECT um.codmedicamento, COUNT(*) medCount FROM UsoMedicamento um
        GROUP BY um.codmedicamento
    );

493

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [8]:
CREATE VIEW NomeCount as
SELECT m.nomeMedicamento, cmc.medCOUNT
    FROM (
        SELECT um.codmedicamento, COUNT(*) medCount FROM UsoMedicamento um
            GROUP BY um.codmedicamento) cmc,
        medicamento m
    WHERE
        cmc.codMedicamento = m.codigo
;

In [9]:
SELECT nc.nomeMedicamento, nc.medCount
    FROM 
        NomeCount nc
    WHERE 
        nc.medCount = (SELECT MAX(aux.medCount) FROM NomeCount aux)
;

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [10]:
SELECT AVG(mediaPessoa) FROM
    (SELECT um.idPessoa, AVG(um.diasUso) mediaPessoa from UsoMedicamento um
        GROUP BY um.idPessoa)
;

2586

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [11]:
SELECT idPessoa, mediaPessoa
    FROM
        (SELECT um.idPessoa, AVG(um.diasUso) mediaPessoa from UsoMedicamento um
            GROUP BY um.idPessoa)
    WHERE 
        mediaPessoa > SELECT AVG(mediaPessoa) 
            FROM
                (SELECT um.idPessoa, AVG(um.diasUso) mediaPessoa from UsoMedicamento um
                    GROUP BY um.idPessoa)
;

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [12]:
--Pessoas que usam acima da media
CREATE VIEW AcimaMedUso AS
SELECT idPessoa, mediaUsoPessoa
    FROM
        (SELECT um.idPessoa, AVG(um.diasUso) mediaUsoPessoa from UsoMedicamento um
            GROUP BY um.idPessoa)
    WHERE 
        mediaUsoPessoa > (SELECT AVG(mediaPessoa) 
            FROM
                (SELECT um.idPessoa, AVG(um.diasUso) mediaPessoa from UsoMedicamento um
                    GROUP BY um.idPessoa))
;

In [13]:
-- Pessoas acima da media de idade
CREATE VIEW AcimaMedIdade AS
SELECT id, idade
    FROM
        Pessoa
    WHERE
        idade > SELECT AVG(idade) FROM Pessoa
;

In [14]:
SELECT COUNT(id) FROM 
    (SELECT ami.id, ami.idade, amu.mediaUsoPessoa
        FROM
            AcimaMedIdade ami, AcimaMedUso amu
        WHERE
            ami.id = amu.idPessoa)
;

SELECT COUNT(id) FROM AcimaMedIdade;

SELECT COUNT(idPessoa) FROM AcimaMedUso;
SELECT COUNT(id) FROM Pessoa;


-- Pessoas que usam mais medicamento do que a media e estão acima da media de idade
-- Pessoas acima da media da idade
-- Pessoas que usam mais medicamento do que a media
-- Total de pessoas

-- Pessoas acima da média utilizam mais medicamento do que a media, proporcionalmente

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [4]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.